In [1]:
import bw2data, bw2calc, bw2io
from edges import EdgeLCIA, get_available_methods
from bw2analyzer.contribution import ContributionAnalysis

In [2]:
bw2data.projects.set_current("ecoinvent-3.10-cutoff")
act = bw2data.Database("ecoinvent-3.10.1-cutoff").random()
#act = [a for a in bw2data.Database("ecoinvent-3.10-cutoff") if "NMC" in #a["name"]][0]
act

'polymethyl methacrylate production, beads' (kilogram, RoW, None)

In [3]:
[m for m in get_available_methods() if "ImpactWorld+" in str(m)]

[('ImpactWorld+ 2.1', 'Freshwater acidification', 'damage'),
 ('ImpactWorld+ 2.1', 'Freshwater acidification', 'midpoint'),
 ('ImpactWorld+ 2.1', 'Freshwater eutrophication', 'damage'),
 ('ImpactWorld+ 2.1', 'Freshwater eutrophication', 'midpoint'),
 ('ImpactWorld+ 2.1', 'Land occupation, biodiversity', 'damage'),
 ('ImpactWorld+ 2.1', 'Land occupation, biodiversity', 'midpoint'),
 ('ImpactWorld+ 2.1', 'Land transformation, biodiversity', 'damage'),
 ('ImpactWorld+ 2.1', 'Land transformation, biodiversity', 'midpoint'),
 ('ImpactWorld+ 2.1', 'Marine eutrophication', 'damage'),
 ('ImpactWorld+ 2.1', 'Marine eutrophication', 'midpoint'),
 ('ImpactWorld+ 2.1', 'Terrestrial acidification', 'damage'),
 ('ImpactWorld+ 2.1', 'Terrestrial acidification', 'midpoint'),
 ('ImpactWorld+ 2.1', 'Thermally polluted water', 'damage'),
 ('ImpactWorld+ 2.1', 'Thermally polluted water', 'midpoint'),
 ('ImpactWorld+ 2.1', 'Water availability, freshwater ecosystem', 'damage'),
 ('ImpactWorld+ 2.1', 'Water 

In [4]:
#method = ('GeoPolRisk', '2024')
# method = ('ImpactWorld+ 2.1', 'Land transformation, biodiversity', 'midpoint')
methods = [m for m in get_available_methods() if "ImpactWorld+" in str(m)][:1]

for method in methods:
  LCA = EdgeLCIA({act: 1}, method)
  LCA.lci()
  LCA.lcia()
  #LCA.score

/opt/homebrew/Caskroom/miniforge/base/envs/bw2/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 4.83e+13)
  warnings.warn(msg, UmfpackWarning)


+-------------------------------------------------------+------------------------------------------------------------+
|                      Method name                      | ('ImpactWorld+ 2.1', 'Freshwater acidification', 'damage') |
|                       Data file                       |      ImpactWorld+ 2.1_Freshwater acidification_damage      |
|                  Unique CFs in method                 |                            4598                            |
|                    Unique CFs used                    |                            731                             |
|               CFs without eligible exc.               |                           18818                            |
| Eligible biosphere exc. characterized/uncharacterized |                         10543 / 0                          |
+-------------------------------------------------------+------------------------------------------------------------+


In [5]:
LCA.generate_cf_table().to_excel("df.xlsx")

Matrix type: biosphere


In [6]:
from constructive_geometries import geomatcher
geo = geomatcher.Geomatcher()

geo.within("IN-UP", biggest_first=False)

[('ecoinvent', 'IN-UP'),
 ('ecoinvent', 'IN-Northern grid'),
 'IN',
 ('ecoinvent', 'SAS'),
 ('ecoinvent', 'IAI Area, Asia, without China and GCC'),
 ('ecoinvent', 'UN-ASIA'),
 ('ecoinvent', 'Asia without China'),
 ('ecoinvent', 'RAS'),
 'GLO']

In [6]:
set([x["supplier"]["name"] for x in LCA.ignored_method_exchanges])

{'Water',
 'Water, cooling, unspecified natural origin',
 'Water, in air',
 'Water, lake',
 'Water, river',
 'Water, turbine use, unspecified natural origin',
 'Water, unspecified natural origin',
 'Water, well, in ground'}

In [7]:
LCA.generate_cf_table()

Matrix type: biosphere


,supplier name,supplier categories,consumer name,consumer reference product,consumer location,amount,CF,impact
0,"Water, cooling, unspecified natural origin","(natural resource, in water)","particleboard production, uncoated, from virgi...","particleboard, uncoated",RoW,6.544631e-11,20.179399,1.320667e-09
1,"Water, cooling, unspecified natural origin","(natural resource, in water)","heat and power co-generation, lignite","electricity, high voltage",SI,1.453419e-07,1.087201,1.580158e-07
2,"Water, cooling, unspecified natural origin","(natural resource, in water)",atrazine production,atrazine,RoW,4.228744e-12,19.819034,8.380961e-11
3,"Water, cooling, unspecified natural origin","(natural resource, in water)","sheet rolling, copper","sheet rolling, copper",RoW,1.290820e-10,19.819034,2.558280e-09
4,"Water, cooling, unspecified natural origin","(natural resource, in water)","phosphoric acid production, dihydrate process","phosphoric acid, fertiliser grade, without wat...",MA,5.123237e-08,54.031045,2.768139e-06
...,...,...,...,...,...,...,...,...
4923,"Water, in air","(natural resource, in air)",soy-based resin production,"polyester resin, unsaturated",US,4.729868e-16,9.087076,4.298067e-15
4924,"Water, in air","(natural resource, in air)",soy-based resin production,"polyester resin, unsaturated",RoW,1.378576e-15,20.175187,2.781303e-14
4925,"Water, in air","(natural resource, in air)",clinker production,clinker,RoW,1.625316e-09,20.627229,3.352577e-08
4926,"Water, in air","(natural resource, in air)","particleboard production, cement bonded","particleboard, cement bonded",RER,8.635607e-19,6.750246,5.829247e-18


In [7]:
LCA.ignored_locations

set()

In [9]:
method = ('AWARE 1.2c', 'Country', 'mixed', 'yearly')
convmethod = (
  # 'ecoinvent-3.10',
  'EF v3.1 EN15804',
  'water use',
  'user deprivation potential (deprivation-weighted water consumption)')

In [10]:
import bw2calc

convLCA = bw2calc.LCA({act: 1}, convmethod)
convLCA.lci()
convLCA.lcia()
convLCA.score

/opt/homebrew/Caskroom/miniforge/base/envs/bw2/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 4.83e+13)
  warnings.warn(msg, UmfpackWarning)


0.06176615560312869